In [2]:
import cv2
import numpy as np
import torch
from pathlib import Path
from PIL import Image
from matplotlib import pyplot as plt
from segment_anything import SamPredictor, sam_model_registry

%matplotlib inline

## Survey on CoCo-stuff

In [3]:
data_dir = '/sda1/Datasets/CoCo'

ann_dir = data_dir + '/annotations'
train_ann_dir = ann_dir + '/train2017'
valid_ann_dir = ann_dir + '/val2017'

img_dir = data_dir + '/images'
train_img_dir = img_dir + '/train2017'
valid_img_dir = img_dir + '/val2017'

In [4]:
print(train_ann_dir)
train_ann_img = sorted(Path(train_ann_dir).glob('*.png'))
print(len(train_ann_img))

/sda1/Datasets/CoCo/annotations/train2017
118287


In [7]:
for path in train_ann_img[:10]:
    mask = Image.open(path).convert('L')
    print(mask.size, np.unique(mask))

(640, 480) [ 50  54  55 120 142 164 255]
(640, 426) [ 24 105 123 125 168 181 255]
(640, 428) [ 63  85 118 123 141 171 255]
(640, 425) [ 23 123 255]
(481, 640) [  0  27 104 147 154 156 168 255]
(640, 478) [ 17 104 114 116 131 171 255]
(381, 500) [  0  18  63 112 123 156 162 168 255]
(640, 488) [  0  21 122 141 255]
(480, 640) [  2   7  12  84  96 102 139 141 148 156 168 171 174 180 255]
(640, 426) [  2   6   7  94  95 114 127 139 143 146 148 150 156 168 255]


## Filter DUTS dataset according to the mask area size

In [7]:
import shutil

img_dir = '/sda1/Datasets/DUTS/DUTS-TE/Std-Image'
mask_dir = '/sda1/Datasets/DUTS/DUTS-TE/Std-Mask'

img_paths = sorted(Path(img_dir).glob('*.jpg'))
mask_paths = sorted(Path(mask_dir).glob('*.png'))

for img_path, mask_path in zip(img_paths, mask_paths):
    img = cv2.imread(str(img_path))
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    mask = mask / 255
    if np.sum(mask) > mask.shape[0] * mask.shape[1] * 0.25:
        shutil.copy(str(img_path), f'/sda1/Datasets/DUTS/DUTS-TE/Std-Image-30/{img_path.name}')
        shutil.copy(str(mask_path), f'/sda1/Datasets/DUTS/DUTS-TE/Std-Mask-30/{mask_path.name}')

## Draw contours with red color

In [7]:
import cv2
import numpy as np

img_path = '/sda1/Datasets/DUTS/DUTS-TE/DUTS-TE-Image/ILSVRC2012_test_00000025.jpg'
mask_path = '/sda1/Datasets/DUTS/DUTS-TE/DUTS-TE-Mask/ILSVRC2012_test_00000025.png'
img = cv2.imread(img_path)
mask = cv2.cvtColor(cv2.imread(mask_path), cv2.COLOR_BGR2GRAY)

mask = cv2.threshold(mask, 127, 1, cv2.THRESH_BINARY)[1]

masked_img = np.stack([mask, mask, mask], axis=-1) * img
cv2.imwrite('/home/chengxin/Documents/Nutstore/Publication/ObjectWatermark/figs/masked_img.png', masked_img, [cv2.IMWRITE_PNG_COMPRESSION, 0])

True